In [1]:
import warnings #игнорирует ошибки
warnings.filterwarnings("ignore")

# Опорные вектора

## Линейный классификатор

Следующий код Scikit-Learn загружает набор данных об ирисе и обучает линейный классификатор SVM для обнаружения цветков ириса виргинского .Конвейер сначала масштабирует функции, а затем использует LinearSVC с параметром регурялизации C=1: 

In [2]:
from sklearn.datasets import load_iris # датасет
from sklearn.pipeline import make_pipeline # контейнер трансформации
from sklearn.preprocessing import StandardScaler #стандартизация признаков
from sklearn.svm import LinearSVC 
 
iris = load_iris(as_frame=True) 
X = iris.data[["petal length (cm)", "petal width (cm)"]].values # преобразование в матрицу
y = (iris.target == 2)  # ирис virginica 
 
svm_clf = make_pipeline(StandardScaler(), 
                        LinearSVC(C=1, random_state=42)) #загружаем модель
svm_clf.fit(X, y) #обучаем
X_new = [[5.5, 1.7], [5.0, 1.5]] #навые признаки для прогноза
svm_clf.predict(X_new) #предсказываем

array([ True, False])

Посмотрим на оценки, которые SVM использовал для этих прогнозов. Они измеряют расстояние со знаком между каждым экземпляром и границей решения:

In [3]:
svm_clf.decision_function(X_new)

array([ 0.66163411, -0.22036063])

В отличие от LogisticRegression, LinearSVC не имеет predict_proba() метода оценки вероятностей класса. Тем не менее, если вы используете SVC класс вместо LinearSVC, и если вы установите его probability гиперпараметр на True, то модель будет соответствовать дополнительной модели в конце обучения, чтобы сопоставить оценки функции принятия решений SVM с оценочными вероятностями. Под капотом это требует использования 5-кратной перекрестной проверки для создания прогнозов вне выборки для каждого экземпляра в обучающем наборе, а затем обучения модели, что значительно замедлит обучение. После этого будут доступны методы predict_proba() и .predict_log_proba() 

## Неленейный классификатор

### Линейная модель с применением полинома

Если данные не ленейны, можно воспользоваться преобразователем PolinomialFeatures (для полиноминальной регрессии). Сгенирируем игрушечные наборы данных, в которых точки данных имеют форму двух чередующихся полумесяцев (реализовать это можно с помощью  make_moons).

In [4]:
from sklearn.datasets import make_moons 
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

In [5]:
X, y = make_moons(n_samples=100, noise=0.15, random_state=42) #генерируем игрушечный набор с данными

In [6]:
polynomial_svm_clf = make_pipeline( 
    PolynomialFeatures(degree=3), 
    StandardScaler(), 
    LinearSVC(C=10, max_iter=10_000, random_state=42) 
) 
polynomial_svm_clf.fit(X, y) 

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=3)),
                ('standardscaler', StandardScaler()),
                ('linearsvc',
                 LinearSVC(C=10, max_iter=10000, random_state=42))])

### Полиномиальное ядро

В модель опорных векторов встроен гиперпараметр, позволяет получить такой же результат, как если бы вы добавили много полиномиальных признаков. это позволяет избежать комбинаторного взрыва кол-ва функций.

In [7]:
from sklearn.svm import SVC 
 
poly_kernel_svm_clf = make_pipeline(StandardScaler(), 
                                    SVC(kernel="poly", degree=3, coef0=1, C=5)) # degree=3 заданная степень, 
#Гиперпараметр coef0 определяет, насколько сильно на модель влияют термины высокой степени по сравнению с терминами низкой степени.
poly_kernel_svm_clf.fit(X, y) 

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=5, coef0=1, kernel='poly'))])

Очевидно, что если ваша модель переоснащается, вы можете уменьшить степень полинома degree. И наоборот, если он недостаточно подходит, вы можете попробовать увеличить его.

### Гауссовское ядро RBF 

Другая техника для решения нелинейных задач - нужно добавить функции, вычисленные с помощью функции сходства, которая измеряет, насколько каждый экземпляр похож на конкретный ориентир. И снова трюк с ядром делает свое волшебство SVM, позволяя получить такой же результат, как если бы вы добавили много признаков подобия, но фактически не сделали этого. Давайте попробуем SVC класс с ядром Gaussian RBF:

In [8]:
rbf_kernel_svm_clf = make_pipeline(StandardScaler(), 
                                   SVC(kernel="rbf", gamma=5, C=0.001)) #гиперпараметр coef0 в данном случае не имеет смысла
rbf_kernel_svm_clf.fit(X, y) 

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=0.001, gamma=5))])

Увеличение gamma делает колоколообразную кривую узкой. В результате диапазон влияния каждого экземпляра меньше: граница решения становится более нерегулярной, колеблясь вокруг отдельных экземпляров. И наоборот, малое gamma значение делает колоколообразную кривую шире: экземпляры имеют больший диапазон влияния, и граница принятия решения становится более гладкой.Таким образом, gamma действует как гиперпараметр регуляризации : если ваша модель переоснащается, вы должны уменьшить gamma ; если он не подходит, вы должны увеличить gamma (аналогично гиперпараметру C).

Как правило, вы всегда должны сначала попробовать линейное ядро. Класс LinearSVC намного быстрее, чем SVC(kernel="linear"), особенно если обучающая выборка очень велика. Если он не слишком велик, вам также следует попробовать ядерные SVM, начиная с ядра Gaussian RBF; это часто работает очень хорошо. Затем, если у вас есть свободное время и вычислительная мощность, вы можете поэкспериментировать с несколькими другими ядрами, используя поиск по гиперпараметрам.

### Сравнение классов Scikit-Learn для классификации SVM 

- LinearSVC:           Временная сложность   О ( м × n ),      Внешняя поддержка - Нет,     Требуется масштабирование -  Да,   Трюк с ядром - Нет. 
- SVC:           Временная сложность    O ( м ² × n )  до O ( м ³ × n ),      Внешняя поддержка - Нет,    Требуется масштабирование - Да,   Трюк с ядром - Да.
- SGDClassifier:           Временная сложность   О ( м × n ),     Внешняя поддержка - Нет,     Требуется масштабирование -  Да,    Трюк с ядром - Нет. 


## SVM-регрессия 

Можно использовать класс Scikit-Learn LinearSVR для выполнения линейной регрессии SVM

In [9]:
import numpy as np
from sklearn.svm import LinearSVR 
#эти 3 строки генерируют простой линейный набор данных
np.random.seed(42)
X = 2 * np.random.rand(50, 1)
y = 4 + 3 * X[:, 0] + np.random.randn(50)
 
svm_reg = make_pipeline(StandardScaler(), 
                        LinearSVR(epsilon=0.5, random_state=42)) 
svm_reg.fit(X, y) 

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvr', LinearSVR(epsilon=0.5, random_state=42))])

Гиперпараметр, характеризующий качество модели регрессии - эпсилон. Уменьшение ϵ увеличивает количество опорных векторов, что упорядочивает модель. Более того, если вы добавите больше обучающих экземпляров в поле, это не повлияет на прогнозы модели; таким образом, модель называется ϵ-нечувствительной.

Следующий код использует класс Scikit-Learn SVR:

In [10]:
from sklearn.svm import SVR 
 
# эти 3 строки генерируют простой квадратичный набор данных
np.random.seed(42)
X = 2 * np.random.rand(50, 1) - 1
y = 0.2 + 0.1 * X[:, 0] + 0.5 * X[:, 0] ** 2 + np.random.randn(50) / 10

svm_poly_reg = make_pipeline(StandardScaler(), 
                             SVR(kernel="poly", degree=2, C=0.01, epsilon=0.1)) 
svm_poly_reg.fit(X, y) 

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svr', SVR(C=0.01, degree=2, kernel='poly'))])

# Самостоятельные задачи

## Обучите классификатор SVM на наборе данных wine

In [11]:
from sklearn.datasets import load_wine
wine = load_wine(as_frame=True)

In [12]:
wine.data

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


In [13]:
wine.target

0      0
1      0
2      0
3      0
4      0
      ..
173    2
174    2
175    2
176    2
177    2
Name: target, Length: 178, dtype: int32

In [14]:
from sklearn.model_selection import train_test_split

In [15]:

X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.3, random_state= 42)

In [16]:
X_test.shape

(54, 13)

In [17]:
X_train.shape

(124, 13)

Начнем с простого, с линейного классификатора SVM. Необходимо помнить, что для модели опорных векторов всегда масштабируем признаки.

In [18]:
lin_clf = make_pipeline(StandardScaler(), LinearSVC(random_state = 42))
lin_clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(random_state=42))])

Оценим базовую модель с помощью cross_val_score:

In [19]:
from sklearn.model_selection import cross_val_score

cross_val_score(lin_clf, X_train, y_train).mean() # по умолчанию пять блоков с перекрсной проверкой

0.984

Теперь, посмотрим на оценку для модели с ядром:

In [20]:
svm_clf = make_pipeline(StandardScaler(), SVC(random_state=42))
cross_val_score(svm_clf, X_train, y_train).mean()

0.9676666666666666

Результат оказался хуже, чем у простой линейной модели. Попробуем подобрать гиперпараметры с помощью RandomizedSearchCV:

In [21]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform, uniform
#добавим параметры регурялизации модели
param_distrib = {
    "svc__gamma": loguniform(0.001, 0.1), # Логарифмическая или обратная непрерывная случайная величина.
    "svc__C": uniform(1, 10)} # Равномерная непрерывная случайная величина.

random_svm_clf = RandomizedSearchCV(svm_clf, param_distrib, n_iter=100, cv=5, random_state=42)
random_svm_clf.fit(X_train, y_train)
random_svm_clf.best_estimator_

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc',
                 SVC(C=8.31993941811405, gamma=0.015751320499779724,
                     random_state=42))])

In [22]:
random_svm_clf.best_score_

0.984

Оценка на обучающей выборке для моделей одинакова. Протестируем их:

In [23]:
lin_clf.score(X_test, y_test)

0.9814814814814815

In [24]:
random_svm_clf.score(X_test, y_test)

0.9814814814814815

Обе модели показали одинаковый результат.

## Обучите и настройте регрессор SVM на наборе данных о жилье в Калифорнии

In [25]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

In [26]:
X = housing.data
y = housing.target

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Линейный регрессор:

In [28]:
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error

In [29]:
lin_reg = make_pipeline(StandardScaler(), LinearSVR(max_iter = 5000, random_state=42)) # добавим итерации для улучшения качества
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
mse

0.9595484665813284

In [30]:
rmse = np.sqrt(mse)
rmse

0.979565447829459

В этом наборе данных квартиры оцениваются в сотни тысяч долларов, поэтому ошибка приблизительно равна 97 956,5 долларов. Это очень много, попробуем применить модель с ядром, для улучшения качества.

In [31]:
from sklearn.svm import SVR

In [32]:
param_distrib = {
    "svr__gamma": loguniform(0.001, 0.1),
    "svr__C": uniform(1, 10)}
svm_reg = make_pipeline(StandardScaler(), SVR())
random_svm_reg = RandomizedSearchCV(svm_reg, param_distrib, n_iter=100, cv=5, random_state=42)
    

In [33]:
random_svm_reg.fit(X_train[:2000], y_train[:2000])

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('standardscaler',
                                              StandardScaler()),
                                             ('svr', SVR())]),
                   n_iter=100,
                   param_distributions={'svr__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000016B4A947EE0>,
                                        'svr__gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000016B4A96BE80>},
                   random_state=42)

In [34]:
random_svm_reg.best_estimator_

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svr', SVR(C=4.63629602379294, gamma=0.08781408196485974))])

In [35]:
- cross_val_score(random_svm_reg.best_estimator_, X_train, y_train, scoring='neg_root_mean_squared_error')

array([0.58835648, 0.57468589, 0.58085278, 0.57109886, 0.59853029])

Качество модели заметно улучшилось. Испытаем модель на тестовой выборке:

In [36]:
y_pred = random_svm_reg.best_estimator_.predict(X_test)
mse = mean_squared_error(y_pred, y_test)
rmse = np.sqrt(mse)
rmse

0.585473226517223